<a href="https://colab.research.google.com/github/Khushi-P-2004/Machine-Learning-Project/blob/main/CommentCategoryPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
X = train_df.drop(["label","created_date"], axis=1)
y = train_df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
test_df=test_df.drop('created_date',axis=1)

In [ ]:
train_df.shape

In [ ]:
train_df.dtypes

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.isnull().mean()*100

In [ ]:
train_df['label'].value_counts()

In [ ]:
train_df['label'].value_counts(normalize=True)

In [ ]:
train_df['comment'].str.len().describe()

In [ ]:
train_df['comment'].str.split().str.len().describe()

In [ ]:
from collections import Counter
Counter(" ".join(train_df['comment'].fillna("")).split()).most_common(20)

In [ ]:
train_df['upvote'].describe()

In [ ]:
train_df['downvote'].describe()

In [ ]:
train_df['comment'] = train_df['comment'].fillna("")

In [ ]:
X_train['comment'] = X_train['comment'].fillna("")
X_test['comment'] = X_test['comment'].fillna("")

In [ ]:
numeric_f = ['upvote','downvote','if_1','if_2']
binary_f = ['emoticon_1','emoticon_2','emoticon_3','race','religion','gender','disability']

tfidf = TfidfVectorizer(max_features=50000,
                        ngram_range=(1,2),
                        stop_words='english',
                        min_df=3,
                        sublinear_tf=True)

prep = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_f),
        ('bin', 'passthrough', ['emoticon_1','emoticon_2','emoticon_3','disability']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['race','religion','gender']),
        ('text', tfidf, 'comment')
    ])

In [ ]:
model_lr = LogisticRegression(
    C=50,
    solver='saga',
    penalty='l2',
    max_iter=5000,
    class_weight='balanced',
    n_jobs=-1
)

pipe_lr = Pipeline([
    ('preprocess', prep),
    ('model_lr', model_lr)
])

pipe_lr.fit(X_train,y_train)
lr_y_pred=pipe_lr.predict(X_test)
f1_score(y_test, lr_y_pred, average='macro')

In [ ]:
#hyperparameter tuning
param_grid_lr = {
    'model_lr__C':[10,20,50]
}
grid_lr = GridSearchCV(
    pipe_lr,
    param_grid=param_grid_lr,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=True
)
grid_lr.fit(X_train,y_train)
print("Best LR params:", grid_lr.best_params_)
print("best score:", grid_lr.best_score_)

In [ ]:
test_df["comment"] = test_df["comment"].fillna("")

In [ ]:
X["comment"] = X["comment"].fillna("")

In [ ]:
pipe_lr.fit(X,y)
test_preds = pipe_lr.predict(test_df)